# Installations

In [ ]:
!pip install nltk spacy emoji

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
pip install sentencepiece nltk

# Commit to GitHub

In [ ]:
!pwd                # shows your current folder
!git status         # check uncommitted changes
!git add .
!git commit -m "Data text Preprocessing and cleaning"
!git push origin main

# Import Statements

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter
import re
from nltk import ngrams
from textblob import TextBlob
import nltk
import re
import emoji
from nltk.tokenize import word_tokenize
import spacy
nltk.download('stopwords')
from nltk.corpus import stopwords
import sentencepiece as spm

stop_words = set(stopwords.words('english'))

from langdetect import detect
import langdetect

nltk.download('punkt')
nltk.download('punkt_tab')


# Initial Preprocessing Steps

In [ ]:
# Load dataset
df_posts = pd.read_csv("Final_Posts_Data.csv")
df_posts.head()

In [ ]:
# Dataset overview
print("Dataset shape:", df_posts.shape)

In [ ]:
# Check for duplicates based on a specific column
duplicates_name = df_posts.duplicated(subset=['content'])
print("\nDuplicates based on 'content':")
print(df_posts[duplicates_name])

In [ ]:
# Missing Values
missing_counts = df_posts.isnull().sum()
print("Missing values per column:\n", missing_counts)

In [ ]:
# Missing Values
missing_counts = df_posts.isnull().sum()
print("Missing values per column:\n", missing_counts) 

In [ ]:
# Fill 'keyword' missing values
df_posts['keyword'] = df_posts['keyword'].fillna('no keyword')

# Fill 'author' missing values
df_posts['author'] = df_posts['author'].fillna('no author')

# Fill 'score' missing values with the median
median_score = df_posts['score'].median()
df_posts['score'] = df_posts['score'].fillna(median_score)

# Fill 'num_comments' missing values with 0 only where source == 'comments'
mask = df_posts['source'] == 'comments'
df_posts.loc[mask, 'num_comments'] = df_posts.loc[mask, 'num_comments'].fillna(0)

# Fill 'parent_post' missing values
df_posts['parent_post'] = df_posts['parent_post'].fillna('no post')

# Drop rows where 'content' is missing (only 1 row)
df_posts = df_posts.dropna(subset=['content'])
# reset the index
df_posts.reset_index(drop=True, inplace=True)

missing_counts = df_posts.isnull().sum()
print("Missing values per column:\n", missing_counts) 

print("\n Dataset shape:", df_posts.shape)

df_posts.head()

In [ ]:
# converting date column into a readable format
df_posts['created_date'] = pd.to_datetime(df_posts['created_utc'], unit='s').dt.date
df_posts['created_time'] = pd.to_datetime(df_posts['created_utc'], unit='s').dt.time

# Drop the original 'created_utc' column
df_posts.drop(columns=['created_utc'], inplace=True)
df_posts


### Drop non-english data (sinhala and tamil)

In [ ]:
def detect_language(text):
    try:
        return detect(text)
    except langdetect.lang_detect_exception.LangDetectException:
        return 'unknown'

# Create a new column for language
df_posts['language'] = df_posts['content'].apply(detect_language)

In [ ]:
# Keep only English posts
df_posts = df_posts[df_posts['language'] == 'en'].copy()

# Drop the language column
df_posts.drop(columns=['language'], inplace=True)
print(df_posts.shape)

df_posts.head()

### **1. Language Filtering**

* [✔️] Detect language of each post.
* [✔️] Remove posts in Sinhala, Tamil, or any non-English languages.

---

### **2. Text Normalization**

* [ ] Convert all text to lowercase.
* [ ] Strip leading and trailing whitespaces.
* [ ] Normalize unicode characters (optional, e.g., accented letters).

---

### **3. Removing Irrelevant Content**

* [ ] Remove URLs (e.g., `http://`, `https://`, `www.`).
* [ ] Remove emojis and emoticons.
* [ ] Remove platform-specific metadata (e.g., `reddit`, `u/username`, `r/subreddit`, `comments`).
* [ ] Remove escape sequences (e.g., `\n`, `\t`).

---

### **4. Handling Special Characters**

* [ ] Remove punctuation marks (`!`, `.`, `,`, `?`, etc.).
* [ ] Remove other non-alphanumeric characters (`@`, `#`, `%`, `^`, etc.).
* [ ] Optionally remove numbers (if not needed for analysis).

---

### **5. Tokenization and Text Structuring**

* [ ] Tokenize text into words (if needed for further analysis).
* [ ] Optionally remove stopwords (`the`, `is`, `and`, etc.).
* [ ] Optionally lemmatize or stem words.

---

### **6. Filtering by Length**

* [ ] Remove posts with fewer than 25 words.
* [ ] Remove posts with more than 1000 words.

---

### **7. Corpus Analysis**

* [ ] Calculate the total number of words in the cleaned dataset.
* [ ] Calculate the number of unique words in the cleaned dataset.

---

✅ **Optional / Advanced Steps**

* [✔️] Handle duplicate posts if any.
* [ ] Correct common typos or spelling errors.
* [ ] Normalize spacing between words.

# Text Cleaning Pipeline

In [ ]:
# Load English model for lemmatization
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Define stopwords
stop_words = set(stopwords.words('english'))

In [ ]:
# Function to replace emojis with textual description
def emoji_to_text(text):
    return emoji.demojize(text, delimiters=(" ", " "))

In [ ]:
# Function to clean text
def clean_text(text):
    # Convert emojis to text
    text = emoji_to_text(text)
    
    # Remove URLs
    text = re.sub(r'http\S+|www\.\S+', '', text)
    
    # Remove Reddit platform metadata (u/username, r/subreddit)
    text = re.sub(r'u\/\w+', '', text)
    text = re.sub(r'r\/\w+', '', text)
    
    # Remove escape sequences and extra whitespace
    text = text.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ')
    text = re.sub(r'\s+', ' ', text)
    
    # Remove punctuation (except within words like can't, won't)
    text = re.sub(r'[^\w\s\']', '', text)
    
    # Lowercase
    text = text.lower()
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove stopwords
    tokens = [t for t in tokens if t not in stop_words]
    
    # Lemmatization
    doc = nlp(" ".join(tokens))
    lemmatized = [token.lemma_ for token in doc]
    
    # Join back into string
    cleaned_text = " ".join(lemmatized)
    
    return cleaned_text

In [ ]:
# Apply cleaning function
df_posts['content_cleaned'] = df_posts['content'].astype(str).apply(clean_text)

In [ ]:
# Filter posts by word count by defining lower and upper treshold values to remove outliers in posts lengths
# min_words = 25
# max_words = 1000

# def word_count_filter(text):
#     count = len(text.split())
#     return min_words <= count <= max_words

# df_posts = df_posts[df_posts['content_cleaned'].apply(word_count_filter)]

# Create a word count column
df_posts["word_count"] = df_posts["content_cleaned"].apply(lambda x: len(str(x).split()))

# 2. Plot histogram of word counts
plt.figure(figsize=(8, 5))
plt.hist(df_posts["word_count"])
plt.xlabel("Word Count per Document")
plt.ylabel("Frequency")
plt.title("Distribution of Word Counts in df_posts['content_cleaned']")
plt.show()

# 3. Compute recommended thresholds using quantiles
lower_threshold = df_posts["word_count"].quantile(0.05)
upper_threshold = df_posts["word_count"].quantile(0.95)

print("Lower Threshold (5th percentile):", lower_threshold)
print("Upper Threshold (95th percentile):", upper_threshold)

In [ ]:
# 12. Report corpus statistics
all_text = " ".join(df_posts['content_cleaned'])
all_words = all_text.split()
total_words = len(all_words)
unique_words = len(set(all_words))

In [ ]:
print(f"Total words in cleaned corpus: {total_words}")
print(f"Unique words in cleaned corpus: {unique_words}") 

In [ ]:
df_posts.head(20)

In [ ]:
df_posts.shape

In [ ]:
import unicodedata

def normalize_unicode(text):
    """
    Normalize unicode characters using NFKC.
    Converts full-width characters, combined characters, and compatibility characters 
    into a consistent canonical form.
    """
    return unicodedata.normalize("NFKC", text)

# Apply normalization
df_posts["content_cleaned"] = (
    df_posts["content_cleaned"]
    .astype(str)
    .apply(normalize_unicode)
)

In [ ]:
df_posts.head(10)

# Tokenization

## Traditional Word-Based

In [ ]:
# Tokenization

# -----------------------------
# 2.1 Word-level tokenizer + lemmatization + stopwords removal
# -----------------------------
def word_tokenizer(text):
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t.isalpha() and t not in stop_words]
    doc = nlp(" ".join(tokens))
    return [token.lemma_ for token in doc]

# -----------------------------
# 2.2 Prepare corpus for SentencePiece
# -----------------------------
corpus_file = "corpus.txt"
with open(corpus_file, "w", encoding="utf-8") as f:
    for t in df_posts["content_cleaned"]:
        f.write(t + "\n")

# -----------------------------
# 2.3 Train BPE tokenizer
# -----------------------------
spm.SentencePieceTrainer.Train(
    f"--input={corpus_file} --model_prefix=bpe --vocab_size=8000 --model_type=bpe"
)
bpe = spm.SentencePieceProcessor()
bpe.load("bpe.model")

# -----------------------------
# 2.4 Train Unigram LM tokenizer
# -----------------------------
spm.SentencePieceTrainer.Train(
    f"--input={corpus_file} --model_prefix=unigram --vocab_size=8000 --model_type=unigram"
)
unigram = spm.SentencePieceProcessor()
unigram.load("unigram.model")


In [ ]:
# Word-level tokenizer

def word_tokenizer(text):
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t.isalpha() and t not in stop_words]
    doc = nlp(" ".join(tokens))
    return [token.lemma_ for token in doc]

In [ ]:
df_posts["processed"] = df_posts["content_cleaned"].apply(word_tokenizer)

## Prepare corpus for SentencePiece

In [ ]:
# Prepare corpus for SentencePiece
corpus_file = "corpus.txt"
with open(corpus_file, "w", encoding="utf-8") as f:
    for t in df_posts["content_cleaned"]:
        f.write(t + "\n")

### BPE Tokenizer

In [ ]:
spm.SentencePieceTrainer.Train(
    f"--input={corpus_file} --model_prefix=bpe --vocab_size=8000 --model_type=bpe"
)
bpe = spm.SentencePieceProcessor()
bpe.load("bpe.model")

### Unigram tokenizer

In [ ]:
spm.SentencePieceTrainer.Train(
    f"--input={corpus_file} --model_prefix=unigram --vocab_size=8000 --model_type=unigram"
)
unigram = spm.SentencePieceProcessor()
unigram.load("unigram.model")

## Compare Tokenization Schemes Impact using Corpus Statistics

Calculating:
- vocabulary size
- average tokens per document
- total tokens
- rare token frequency (<3 occurrences)
- OOV rate (for word-based tokenizers only)

In [ ]:
# CORPUS STATISTICS PER TOKENIZER

def compute_stats(tokenizer_fn, tokenizer_name):
    all_tokens = []
    doc_lengths = []

    for text in df_posts["content_cleaned"]:
        tokens = tokenizer_fn(text)
        all_tokens.extend(tokens)
        doc_lengths.append(len(tokens))

    vocab = set(all_tokens)
    counter = collections.Counter(all_tokens)

    return {
        "Tokenizer": tokenizer_name,
        "Vocabulary Size": len(vocab),
        "Total Tokens": len(all_tokens),
        "Avg Tokens per Doc": sum(doc_lengths)/len(doc_lengths),
        "Rare Tokens (<3)": sum(1 for t,c in counter.items() if c < 3)
    }

stats_word = compute_stats(word_tokenizer, "Word+Lemmatization")
stats_bpe = compute_stats(lambda t: bpe.encode(t, out_type=str), "BPE")
stats_unigram = compute_stats(lambda t: unigram.encode(t, out_type=str), "Unigram LM")

stats_df = pd.DataFrame([stats_word, stats_bpe, stats_unigram])

In [ ]:
stats_df

In [ ]:
# Set index to Tokenizer (for easier plotting)
plot_df = stats_df.set_index("Tokenizer")

# Plot each metric
plot_df.plot(kind="bar", figsize=(10,6))
plt.title("Comparison of Tokenizer Statistics")
plt.xlabel("Tokenizer")
plt.ylabel("Value")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
# LSTM LANGUAGE MODEL

# -----------------------------
# 4.1 Dataset class
# -----------------------------
class LMDataset(Dataset):
    def __init__(self, token_ids, seq_len=30):
        self.seq_len = seq_len
        self.data = token_ids

    def __len__(self):
        return len(self.data) - self.seq_len

    def __getitem__(self, idx):
        x = torch.tensor(self.data[idx:idx+self.seq_len])
        y = torch.tensor(self.data[idx+1:idx+1+self.seq_len])
        return x, y

# -----------------------------
# 4.2 LSTM LM model
# -----------------------------
class LSTMLM(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        e = self.embed(x)
        o, _ = self.lstm(e)
        return self.fc(o)

# -----------------------------
# 4.3 Training function
# -----------------------------
def train_lm(model, dataloader, epochs=2):
    model.train()
    optim = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        for x, y in tqdm(dataloader, desc=f"Epoch {epoch+1}"):
            optim.zero_grad()
            pred = model(x)
            loss = loss_fn(pred.view(-1, pred.size(-1)), y.view(-1))
            loss.backward()
            optim.step()
    return model

# -----------------------------
# 4.4 Perplexity
# -----------------------------
def perplexity(model, dataloader):
    model.eval()
    loss_fn = nn.CrossEntropyLoss(reduction='sum')
    total_loss, total_tokens = 0, 0

    with torch.no_grad():
        for x, y in dataloader:
            pred = model(x)
            loss = loss_fn(pred.view(-1, pred.size(-1)), y.view(-1))
            total_loss += loss.item()
            total_tokens += y.numel()

    return math.exp(total_loss / total_tokens)

In [ ]:
# TRAIN LSTMs PER TOKENIZER

SEQ, BATCH = 30, 32

# -------- Word-level ----------
word_vocab = {w:i for i,w in enumerate(set(itertools.chain.from_iterable(
    df_posts["content_cleaned"].apply(word_tokenizer)
)))}
word_id = lambda toks: [word_vocab[t] for t in toks if t in word_vocab]

word_ids = list(itertools.chain.from_iterable(
    df_posts["content_cleaned"].apply(word_tokenizer).apply(word_id)
))
word_dl = DataLoader(LMDataset(word_ids, SEQ), batch_size=BATCH)
lm_word = LSTMLM(len(word_vocab))
lm_word = train_lm(lm_word, word_dl)
pp_word = perplexity(lm_word, word_dl)

# -------- BPE ----------
bpe_ids = [i for t in df_posts["content_cleaned"] for i in bpe.encode(t)]
bpe_dl = DataLoader(LMDataset(bpe_ids, SEQ), batch_size=BATCH)
lm_bpe = LSTMLM(bpe.get_piece_size())
lm_bpe = train_lm(lm_bpe, bpe_dl)
pp_bpe = perplexity(lm_bpe, bpe_dl)

# -------- Unigram ----------
uni_ids = [i for t in df_posts["content_cleaned"] for i in unigram.encode(t)]
uni_dl = DataLoader(LMDataset(uni_ids, SEQ), batch_size=BATCH)
lm_uni = LSTMLM(unigram.get_piece_size())
lm_uni = train_lm(lm_uni, uni_dl)
pp_uni = perplexity(lm_uni, uni_dl)

In [ ]:
# FINAL COMPARISON TABLE

final_results = pd.DataFrame({
    "Tokenizer": ["Word+Lemmatization", "BPE", "Unigram"],
    "Vocabulary Size": [stats_word["Vocabulary Size"], stats_bpe["Vocabulary Size"], stats_unigram["Vocabulary Size"]],
    "Total Tokens": [stats_word["Total Tokens"], stats_bpe["Total Tokens"], stats_unigram["Total Tokens"]],
    "Avg Tokens per Doc": [stats_word["Avg Tokens per Doc"], stats_bpe["Avg Tokens per Doc"], stats_unigram["Avg Tokens per Doc"]],
    "Rare Tokens (<3)": [stats_word["Rare Tokens (<3)"], stats_bpe["Rare Tokens (<3)"], stats_unigram["Rare Tokens (<3)"]],
    "Perplexity": [pp_word, pp_bpe, pp_uni]
})
print(final_results)

## ✅ PIPELINE EXPLANATION

Phase 1 — Cleaning

Remove non-English posts

Normalize unicode

Remove URLs, emojis → text

Remove Reddit metadata, punctuation, stopwords

Lowercase & strip

Filter posts <25 or >1000 words

Phase 2 — Tokenization

Word-level + lemmatization

SentencePiece BPE

SentencePiece Unigram

Phase 3 — Corpus Statistics

Compute vocab size, total tokens, avg tokens/doc, rare tokens

Phase 4 — LSTM LM

Build LSTM language model

Train on each tokenization scheme

Phase 5 — Train LSTM + Perplexity

Compare model performance

Phase 6 — Final Table

Compare vocab, tokens, and perplexity

Recommend best tokenizer (likely BPE)